## Installing

In [ ]:
!pip install lasagna-ai

### Check your version!

In [1]:
import lasagna

print(lasagna.__version__)

0.14.0


In [2]:
def foo(a: int, b: int) -> int:
    return a + b

Lorim ipsum

Lorim ipsum

Lorim ipsum

hi


### Hello Lasagna

In [3]:
# TODO